In [132]:
import pandas as pd
from pathlib import Path
from Bio import AlignIO
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Blast import NCBIWWW, NCBIXML
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import re

from src.sup_data_to_fasta import load_xlsx

sup5_file_path = Path("data/Liu_sup5_data.xlsx")
alignments_files_paths = [
    Path("output/EP-OD0_alignments_results.tsv"),
    Path("output/ESP-OD2_alignments_results.tsv"),
    Path("output/SP-OD2_alignments_results.tsv")
    ]

# Análise Exploratória dos Dados Suplementares 5

## Sobre

Este notebook contém uma análise exploratória dos dados fornecidos por Liu, et al. No [suplemento 5](https://www.nature.com/articles/s41467-023-43632-1#additional-information
) com o objetivo de averiguar os tratamentos executados no artigo

In [133]:
def add_scenario_column(df_dict):
    return {k:df.assign(scenario=k) for k,df in df_dict.items()}

def extract_scenario_from_file_path(string_arg):
    return re.search(r"(.*)-(.*)", string_arg).group(1)

def concat_df_dict(df_dict):
    return pd.concat(df_dict.values())

alignments_df_list = [pd.read_csv(file_path, sep='\t') for file_path in alignments_files_paths]
alignments_df_dict = {extract_scenario_from_file_path(k.name):v for k,v in zip(alignments_files_paths, alignments_df_list)}
alignments_df_dict = add_scenario_column(alignments_df_dict)
alignments_df = concat_df_dict(alignments_df_dict)

sup5_data_df_dict = load_xlsx(sup5_file_path)
sup5_data_df_dict = {extract_scenario_from_file_path(k):v for k,v in sup5_data_df_dict.items()}
sup5_data_df_dict = add_scenario_column(sup5_data_df_dict)
sup5_data_df = concat_df_dict(sup5_data_df_dict)

In [134]:
alignments_df.head()

,qseqid,qgi,qacc,qaccver,qlen,sseqid,sallseqid,sgi,sallgi,sacc,...,sscinames,scomnames,sblastnames,sskingdoms,stitle,salltitles,sstrand,qcovs,qcovhsp,scenario
0,rpl19(SL1344_2646),0,rpl19(SL1344_2646),rpl19(SL1344_2646),190,NC_003197.2,NC_003197.2,0,0,NC_003197.2,...,NaN,NaN,NaN,NaN,NC_003197.2 Salmonella enterica subsp. enteric...,NC_003197.2 Salmonella enterica subsp. enteric...,minus,100,100,EP
1,rpl19(SL1344_2646),0,rpl19(SL1344_2646),rpl19(SL1344_2646),189,NC_003197.2,NC_003197.2,0,0,NC_003197.2,...,NaN,NaN,NaN,NaN,NC_003197.2 Salmonella enterica subsp. enteric...,NC_003197.2 Salmonella enterica subsp. enteric...,minus,100,100,EP
2,STnc2180(ncRNA0291),0,STnc2180(ncRNA0291),STnc2180(ncRNA0291),44,NC_003197.2,NC_003197.2,0,0,NC_003197.2,...,NaN,NaN,NaN,NaN,NC_003197.2 Salmonella enterica subsp. enteric...,NC_003197.2 Salmonella enterica subsp. enteric...,minus,100,100,EP
3,SdsR(ncRNA0070),0,SdsR(ncRNA0070),SdsR(ncRNA0070),78,NC_003197.2,NC_003197.2,0,0,NC_003197.2,...,NaN,NaN,NaN,NaN,NC_003197.2 Salmonella enterica subsp. enteric...,NC_003197.2 Salmonella enterica subsp. enteric...,minus,100,100,EP
4,ArcZ(ncRNA0002),0,ArcZ(ncRNA0002),ArcZ(ncRNA0002),59,NC_003197.2,NC_003197.2,0,0,NC_003197.2,...,NaN,NaN,NaN,NaN,NC_003197.2 Salmonella enterica subsp. enteric...,NC_003197.2 Salmonella enterica subsp. enteric...,plus,100,100,EP


In [137]:
def should_be_only_one_genome(df):
    return df.sseqid.nunique() == 1

def query_start_should_always_be_less_than_query_end(df):
    return (df.qstart <= df.qend).all()

# validationg
assert should_be_only_one_genome(alignments_df), "o banco de dados deveria ter apenas um genoma de referência de Salmonella" # all alignments are done to same Salmonella enterica genome
assert query_start_should_always_be_less_than_query_end(alignments_df), "O ínicio da query deve ser menor ou igual ao fim da query"

In [138]:
def not_aligned(col):
    return col[~col.isin(alignments_df.Query_ID)]

RNA1_not_aligned = not_aligned(sup5_data_df["RNA1 name"])
RNA2_not_aligned = not_aligned(sup5_data_df["RNA2 name"])

filtered_alignments_df = alignments_df[alignments_df['qseqid'].isin(RNA1_not_aligned) & alignments_df['Query_ID'].isin(RNA2_not_aligned)]
print(f"# queries não alinhadas: {len(filtered_alignments_df)}")

AttributeError: 'DataFrame' object has no attribute 'Query_ID'

In [42]:
from Bio import SeqIO
from pathlib import Path
import gzip

# Define the path to the genome file
genome_file_path = Path("data/salmonella_genome.fna")

# Open the gzipped file and parse the sequences
with open(genome_file_path, "rt") as file:
    sequences = SeqIO.parse(file, "fasta")
    seqs = list(sequences)
    salmonella_genome = seqs[0]

In [121]:
# use Subject_Start and End to retrieve the sequence from the genome

alignments_df["Subject_Seq"] = alignments_df.get(["Subject_Start", "Subject_End"]).apply(lambda r: salmonella_genome.seq[r[0]-1:r[1]], axis=1)

/tmp/ipykernel_118566/308561010.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alignments_df["Subject_Seq"] = alignments_df.get(["Subject_Start", "Subject_End"]).apply(lambda r: salmonella_genome.seq[r[0]-1:r[1]], axis=1)


In [127]:
len(salmonella_genome)

4857450

In [128]:
1248179 - 1248137

42

In [126]:
alignments_df

,Query_ID,Subject_ID,PIdentity,Alignment_Length,Mismatches,Gap_Openings,Query_Start,Query_End,Subject_Start,Subject_End,E_value,Bit_Score,scenario,Subject_Seq
0,ArcZ(ncRNA0002),NC_003197.2,100.0,58,0,0,1,58,3490451,3490508,5.090000e-25,108.0,EP,"(A, T, T, T, C, C, C, T, G, G, T, G, T, T, G, ..."
1,STnc2110(ncRNA0286),NC_003197.2,100.0,49,0,0,1,49,4009394,4009346,3.940000e-20,91.6,EP,()
2,CpxQ(ncRNA0205),NC_003197.2,100.0,61,0,0,1,61,4271116,4271176,1.180000e-26,113.0,EP,"(G, T, T, T, T, C, C, T, T, G, C, C, A, T, A, ..."
3,STnc2000(ncRNA0278),NC_003197.2,100.0,43,0,0,1,43,1248179,1248137,7.110000e-17,80.5,EP,()
4,ArcZ(ncRNA0002),NC_003197.2,100.0,58,0,0,1,58,3490451,3490508,5.090000e-25,108.0,EP,"(A, T, T, T, C, C, C, T, G, G, T, G, T, T, G, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3302,CpxQ(ncRNA0205),NC_003197.2,100.0,60,0,0,1,60,4271116,4271175,4.130000e-26,111.0,SP,"(G, T, T, T, T, C, C, T, T, G, C, C, A, T, A, ..."
3303,RprA(ncRNA0058),NC_003197.2,100.0,51,0,0,1,51,1444873,1444823,3.250000e-21,95.3,SP,()
3304,CpxQ(ncRNA0205),NC_003197.2,100.0,60,0,0,1,60,4271117,4271176,4.130000e-26,111.0,SP,"(T, T, T, T, C, C, T, T, G, C, C, A, T, A, G, ..."
3305,RprA(ncRNA0058),NC_003197.2,100.0,74,0,0,1,74,1444894,1444821,9.150000e-34,137.0,SP,()


In [125]:
salmonella_genome_sz = len(salmonella_genome)

In [123]:
no_mismatch_alignments = alignments_df[alignments_df["PIdentity"] == 100]

In [129]:
RNA1_seq_matches = no_mismatch_alignments["Subject_Seq"].isin(sup5_data_df["RNA1 seq"])
RNA2_seq_matches = no_mismatch_alignments["Subject_Seq"].isin(sup5_data_df["RNA2 seq"])

not_matched = no_mismatch_alignments[~(RNA1_seq_matches | RNA2_seq_matches)]

In [131]:
not_matched.Subject_Seq.map(len).value_counts()

Subject_Seq
0      2312
72       15
84       13
52       12
57       11
79        9
54        9
55        8
81        6
85        5
56        5
87        4
46        4
86        4
48        3
28        3
64        2
68        2
53        2
74        2
47        2
80        2
73        2
100       2
99        2
59        2
71        2
66        1
89        1
37        1
45        1
36        1
29        1
65        1
42        1
41        1
69        1
92        1
90        1
75        1
31        1
Name: count, dtype: int64